In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

import collections
import madmom
import numpy as np
import scipy
import pretty_midi

from sys import argv
from os.path import exists


ImportError: cannot import name 'MutableSequence' from 'collections' (/opt/anaconda3/envs/name_of_my_env/lib/python3.10/collections/__init__.py)

In [10]:
# x, sr = librosa.load('../ThePowersThatBMP3/09_On_GP.mp3')
# assign directory
directory = '../Discography/Original/' 

# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        # p = subprocess.call(["python chroma_to_midi.py",f,f[-4]+'.png'])
        # lines = ['Readme', 'How to write text files in Python']
        with open('filepaths.txt', 'w') as fp:
            fp.write(f)
            fp.write('\n')

In [27]:
albums = os.listdir('../Discography/Original/')
albums.remove('.DS_Store')
# albums.remove('chromagrams')
print(albums)
chromas = []
with open('filepaths.txt', 'w') as fp:
    for album in albums:
    # print(os.listdir('../Discography/'+album+'/'))
        for f in os.listdir('../Discography/Original/'+album+'/'):
            if f[-3:] == 'mp3' and os.path.isfile(os.path.join('../Discography/Original/'+album+'/', f)):
                # x, sr = librosa.load(os.path.join('../Discography/Original/'+album+'/', f))
                fp.write(os.path.join('../Discography/Original/'+album+'/', f))
                fp.write('\n')

['Exmilitary', 'ThePowersThatB', 'GovernmentPlates', 'TheMoneyStore', 'NoLoveDeepWeb', 'Steroids', 'YearOfTheSnitch', 'BottomlessPit']


In [31]:
fp = open('filepaths.txt', 'r')
with open('filenames.txt', 'w') as fn:
    for l in fp:
        fn.write(l[:-5]+'.midi')
        fn.write('\n')

In [32]:
fp = open('filepaths.txt', 'r')
# fn = open('filenames.txt', 'r')

with open('commands.txt', 'w') as cm:
    for l in fp:
        cm.write('python chroma_to_midi.py ' + l + ' ' + l[:-5]+'.midi 200\n')

In [ ]:
fp = open('filepaths.txt', 'r')
# fn = open('filenames.txt', 'r')

with open('filemaker.txt', 'w') as fm:
    for l in fp:
        fm.write('touch '+ l[:-5]+'.midi\n')

In [21]:
"""
create a chromagram from an audio file and then render that chromagram to MIDI
""" 

# Run the script from the command line
script, path_to_audio, path_to_midi, tempo_var = argv

# Create the chromagram
dcp = madmom.audio.chroma.DeepChromaProcessor()
chroma = dcp(path_to_audio)

# Function to threshhold the chroma array to only the top 3 strongest pitch classes. 
def threshhold_chroma(chromagram):
	# Create an array of zeros the same size/dimension as the chromagram
    chromagram_out = scipy.zeros_like(chromagram)  
    # Loop through the chroma_vector the size of the zeros array and sort for the strongest pitch centers
    for i, chroma_vector in enumerate(chromagram):
        chromagram_out[i, chroma_vector.argsort()[::-1][:3]] = [10, 9, 8]
   
    return chromagram_out

# Call the threshholding function
chroma_out = threshhold_chroma(chroma)

# Function to create our MIDI file based on the above sorted chroma array
def chroma_to_midi(chromagram):

    chroma_midi = pretty_midi.PrettyMIDI(midi_file=None, resolution=220, initial_tempo=int(tempo_var))
    piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)
    
    start=0.0
    end=0.0
    make_a_chord = 0
    vi = 0
    last_vector = []

    # Loop through the chroma vectors of the array by their vector_index
    for vector_index, chroma_vector in enumerate(chromagram):
    	# Everytime we loop through, increase the point in the MIDI file that we are writing by the framesize of the original chromagram
        end+=0.1
        # If it's the first note of the song, or the tonal centers' balance have changed, increase the vector index counter and tell the script we want to make a new chord
        if vi == 0 or not np.array_equal(chroma_vector,last_vector):
            make_a_chord = 1
            vi+=1 
        # If the tonal centers have not changed, don't 
        else:
            make_a_chord = 0
            vi+=1 

        if make_a_chord == 1:
        	# Loop through the current vector of chroma_notes by their chroma_note_index
            for chroma_note_index, chroma_note in enumerate(last_vector):
            	# If it's the strongest chroma center, make a 127 velocity note and append to the pretty_midi object
                if chroma_note == 10:
                    note = pretty_midi.Note(velocity=127, pitch=(chroma_note_index+60), start=start, end=end)
                    piano.notes.append(note)
                # If it's the second strongest chroma center, make a 64 velocity note and append to the pretty_midi object       
                elif chroma_note == 9:
                    note = pretty_midi.Note(velocity=64, pitch=(chroma_note_index+60), start=start, end=end)
                    piano.notes.append(note)   
                # If it's the third strongest chroma center, make a 1 velocity note and append to the pretty_midi object           
                elif chroma_note == 8:
                    note = pretty_midi.Note(velocity=1, pitch=(chroma_note_index+60), start=start, end=end)
                    piano.notes.append(note)   
                else:
                    pass
            # Set the start of the next time we write a note to the end time
            start=end                
        else:
            pass
        # Set the last_vector, so that the next time we loop through we have a state to compare to and decide whether or not we need to write a new MIDI chord
        last_vector = chroma_vector
    # Add the MIDI we just made to the piano instrument   
    chroma_midi.instruments.append(piano)
    # Write the MIDI file to a filename we specified above
    chroma_midi.write(path_to_midi)
    return chroma_midi

# Run the function    
chroma_to_midi(chroma_out)

ImportError: cannot import name 'MutableSequence' from 'collections' (/opt/anaconda3/envs/name_of_my_env/lib/python3.10/collections/__init__.py)

In [22]:
fp = open('filepaths.txt', 'r')
for p in fp:
    path_to_audio = p
    path_to_midi = p[-4]+'.midi'
    tempo_var = 200
    dcp = madmom.audio.chroma.DeepChromaProcessor()
    chroma = dcp(path_to_audio)
    chroma_out = threshhold_chroma(chroma)
    chroma_to_midi(chroma_out)

NameError: name 'madmom' is not defined